# Initialisation

In [1]:
!pip install netCDF4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.5 MB/s eta 0:00:00


In [3]:
!rm -r avg_season

rm: cannot remove 'avg_season': No such file or directory


In [17]:
!rm *.nc
!rm -r season_avg_data_files

In [5]:
def ref():
    !rm *.nc
    !rm -r season_avg_data_files

    # make directory for each season

    !mkdir season_avg_data_files

    !mkdir season_avg_data_files/DJF_avg


    !mkdir season_avg_data_files/MAM_avg


    !mkdir season_avg_data_files/JJA_avg


    !mkdir season_avg_data_files/SON_avg

In [19]:
import shutil

# Create a zip archive of the 'avg_season' folder
shutil.make_archive('avg_season_archive', 'zip', '.', 'avg_season')



'/content/avg_season_archive.zip'

In [18]:
ref()

rm: cannot remove '*.nc': No such file or directory
rm: cannot remove 'season_avg_data_files': No such file or directory


<!--  -->
# Average Functions

In [7]:
import netCDF4 as nc
import numpy as np
import requests
import os
import glob


def calculate_and_save_seasonal_averages(urls):
    ref()
    print("cleared!")

    for url_group in urls:
        n = len(url_group)
        for i in range(n):
            url = url_group[i]
            response = requests.get(url)
            filename = url.split('/')[-1]
            with open(filename, 'wb') as f:
                f.write(response.content)
                print(f"{filename} download done")

    files = glob.glob("*.nc")
    print(files)
    # ds = nc.Dataset("/content/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19500101-19591231.nc")
    for file in files:
        ds = nc.Dataset(file)
        season_data = ds.variables['zmla']
        lat = ds.variables['lat'][:]
        lon = ds.variables['lon'][:]

        # print(season_data)
        # Get the time variable
        time_var = ds['time']

        # Convert numeric time values to datetime objects
        time_values = time_var[:]
        time_units = time_var.units
        calendar = time_var.calendar if hasattr(
            time_var, 'calendar') else 'standard'

        dates = nc.num2date(time_values, units=time_units, calendar=calendar)

        # Define the start and end months for each season (assuming a calendar year)
        seasons = {
            'DJF': (12, 1, 2),
            'MAM': (3, 4, 5),
            'JJA': (6, 7, 8),
            'SON': (9, 10, 11)
        }

        # Loop through each season and compute the average
        for season, (start_month, mid_month, end_month) in seasons.items():

            # Find indices of dates within the current season
            season_indices = [
                i for i, date in enumerate(dates) if (date.month == start_month or date.month == mid_month or date.month == end_month)
            ]
            season_data = ds.variables['zmla'][season_indices, :, :]

            # Calculate the seasonal average
            seasonal_avg = season_data.mean(axis=0)

            # Create a new netCDF file for the seasonal average
            filename = file

            # create the new file within season name folder having that season's average
            season_avg_file = f'season_avg_data_files/{season}_avg/{filename}'

            ncfile = nc.Dataset(season_avg_file, mode='w', format='NETCDF4')

            dim1 = ncfile.createDimension('dim1', seasonal_avg.shape[0])
            dim2 = ncfile.createDimension('dim2', seasonal_avg.shape[1])

            dim3 = ncfile.createDimension('dim3', lat.shape[0])
            dim4 = ncfile.createDimension('dim4', lon.shape[0])

            var = ncfile.createVariable('average', 'f4', ('dim1', 'dim2'))
            varlat = ncfile.createVariable('lat', 'f4', dim3)
            varlot = ncfile.createVariable('lon', 'f4', dim4)
            # latitude = ncfile.createVariable('lat', 'f4', ('dim1', 'dim2', 'dim3'))

            # write the data to the variable
            var[:] = seasonal_avg[:]
            varlat[:] = lat[:]
            varlot[:] = lon[:]

            # Close the new netCDF file
            ncfile.close()

        # Close the original netCDF file
        ds.close()

    # remove downloaded files
    !rm * .nc
    print("avg1 done")
    calc_average_seasonal_files()


def calc_average_seasonal_files():
    # Create a dictionary to store seasonal averages
    seasonal_averages = {
        'DJF': [],
        'MAM': [],
        'JJA': [],
        'SON': [],
    }

    # Loop through the seasons and calculate the average of files
    for season in seasonal_averages.keys():
        season_files = glob.glob(f"season_avg_data_files/{season}_avg/*.nc")

        if not season_files:
            print(f"No files found for {season} season.")
            continue

        # Create an array to store seasonal averages for this season
        season_averages = []

        for season_file in season_files:
            ds = nc.Dataset(season_file)
            seasonal_avg = ds.variables['average'][:]
            season_averages.append(seasonal_avg)
            lat = ds.variables['lat'][:]
            lon = ds.variables['lon'][:]
            ds.close()

        # Calculate the average of seasonal averages
        average_seasonal_avg = np.mean(season_averages, axis=0)

        # Create the output directory if it doesn't exist
        output_dir = f'avg_season/{season}_avg'
        os.makedirs(output_dir, exist_ok=True)

        # Extract the filename from the original file's path
        original_filename = os.path.basename(season_files[0])
        parts = original_filename.split('_')
        filename = '_'.join(parts[:3])
        filename = filename + ".nc"

        # Create a new netCDF file for the average seasonal average
        avg_season_file = os.path.join(output_dir, filename)


        ncfile = nc.Dataset(avg_season_file, mode='w', format='NETCDF4')

        dim1 = ncfile.createDimension('dim1', average_seasonal_avg.shape[0])
        dim2 = ncfile.createDimension('dim2', average_seasonal_avg.shape[1])
        dim3 = ncfile.createDimension('dim3', lat.shape[0])
        dim4 = ncfile.createDimension('dim4', lon.shape[0])

        var = ncfile.createVariable('average', 'f4', ('dim1', 'dim2'))
        var[:] = average_seasonal_avg[:]

        varlat = ncfile.createVariable('lat', 'f4', 'dim3')
        varlat[:] = lat

        varlon = ncfile.createVariable('lon', 'f4', 'dim4')
        varlon[:] = lon

        ncfile.close()

        print(f"Average of {season} season saved to {avg_season_file}")

        # Clean up temporary seasonal average files
        for season_file in season_files:
            os.remove(season_file)

In [13]:
# CESM2 ! 1
urls1 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190326/zmla_Eday_CESM2_amip_r1i1p1f1_gn_20100101-20150101.nc"
    ]

urls2 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r2i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r2i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r2i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r2i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r2i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r2i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r2i1p1f1_gn_20100101-20150101.nc"
    ]

urls3 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_20100101-20150101.nc"
    ]

urls4 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_20100101-20150101.nc"
    ]

urls5 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_20100101-20150101.nc"
    ]
urls6 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_20100101-20150101.nc"
    ]
urls7 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_20100101-20150101.nc"
    ]
urls8 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_20100101-20150101.nc"
    ]
urls9 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_20100101-20150101.nc"
    ]
urls10 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_20100101-20150101.nc"
    ]


urls_1 = [urls1,urls2,urls3,urls4,urls5,urls6,urls7,urls7,urls8,urls9,urls10]
calculate_and_save_seasonal_averages(urls_1)


rm: cannot remove '*.nc': No such file or directory
cleared!
zmla_Eday_CESM2_amip_r1i1p1f1_gn_19500101-19591231.nc download done
zmla_Eday_CESM2_amip_r1i1p1f1_gn_19600101-19691231.nc download done
zmla_Eday_CESM2_amip_r1i1p1f1_gn_19700101-19791231.nc download done
zmla_Eday_CESM2_amip_r1i1p1f1_gn_19800101-19891231.nc download done
zmla_Eday_CESM2_amip_r1i1p1f1_gn_19900101-19991231.nc download done
zmla_Eday_CESM2_amip_r1i1p1f1_gn_20000101-20091231.nc download done
zmla_Eday_CESM2_amip_r1i1p1f1_gn_20100101-20150101.nc download done
zmla_Eday_CESM2_amip_r2i1p1f1_gn_19500101-19591231.nc download done
zmla_Eday_CESM2_amip_r2i1p1f1_gn_19600101-19691231.nc download done
zmla_Eday_CESM2_amip_r2i1p1f1_gn_19700101-19791231.nc download done
zmla_Eday_CESM2_amip_r2i1p1f1_gn_19800101-19891231.nc download done
zmla_Eday_CESM2_amip_r2i1p1f1_gn_19900101-19991231.nc download done
zmla_Eday_CESM2_amip_r2i1p1f1_gn_20000101-20091231.nc download done
zmla_Eday_CESM2_amip_r2i1p1f1_gn_20100101-20150101.nc d

<ipython-input-7-6e9dab23d888>:58: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  season_data = ds.variables['zmla'][season_indices, :, :]


OSError: ignored

In [8]:
# CESM2 FV2 ! 2

urls21 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_20100101-20150101.nc"
    ]
urls22 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_20100101-20150101.nc"
    ]
urls23 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_20100101-20150101.nc"
    ]
urls24 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_20100101-20150103.nc"
    ]
urls2 = [urls21,urls22,urls23]

calculate_and_save_seasonal_averages(urls2)

rm: cannot remove '*.nc': No such file or directory
cleared!
zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19500101-19591231.nc download done
zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19600101-19691231.nc download done
zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19700101-19791231.nc download done
zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19800101-19891231.nc download done
zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19900101-19991231.nc download done
zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_20000101-20091231.nc download done
zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_20100101-20150101.nc download done
zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19500101-19591231.nc download done
zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19600101-19691231.nc download done
zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19700101-19791231.nc download done
zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19800101-19891231.nc download done
zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19900101-19991231.nc download done
zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_20000101-20091231.nc download done
zml

<ipython-input-7-6e9dab23d888>:58: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  season_data = ds.variables['zmla'][season_indices, :, :]


rm: cannot remove 'sample_data': Is a directory
rm: cannot remove 'season_avg_data_files': Is a directory
rm: cannot remove '.nc': No such file or directory
avg1 done
Average of DJF season saved to avg_season/DJF_avg/zmla_Eday_CESM2-FV2.nc
Average of MAM season saved to avg_season/MAM_avg/zmla_Eday_CESM2-FV2.nc
Average of JJA season saved to avg_season/JJA_avg/zmla_Eday_CESM2-FV2.nc
Average of SON season saved to avg_season/SON_avg/zmla_Eday_CESM2-FV2.nc


In [9]:
# CESM2 WACCM ! 3
urls31 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_20100101-20150116.nc"
    ]
urls32 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_20100101-20150101.nc"
    ]

urls33 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_20100101-20150101.nc"
    ]

urls3 = [urls31,urls32,urls33]

calculate_and_save_seasonal_averages(urls3)

rm: cannot remove '*.nc': No such file or directory
cleared!
zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19500101-19591231.nc download done
zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19600101-19691231.nc download done
zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19700101-19791231.nc download done
zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19800101-19891231.nc download done
zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19900101-19991231.nc download done
zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_20000101-20091231.nc download done
zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_20100101-20150116.nc download done
zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19500101-19591231.nc download done
zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19600101-19691231.nc download done
zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19700101-19791231.nc download done
zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19800101-19891231.nc download done
zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19900101-19991231.nc download done
zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_20000101-200

<ipython-input-7-6e9dab23d888>:58: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  season_data = ds.variables['zmla'][season_indices, :, :]


rm: cannot remove 'avg_season': Is a directory
rm: cannot remove 'sample_data': Is a directory
rm: cannot remove 'season_avg_data_files': Is a directory
rm: cannot remove '.nc': No such file or directory
avg1 done
Average of DJF season saved to avg_season/DJF_avg/zmla_Eday_CESM2-WACCM.nc
Average of MAM season saved to avg_season/MAM_avg/zmla_Eday_CESM2-WACCM.nc
Average of JJA season saved to avg_season/JJA_avg/zmla_Eday_CESM2-WACCM.nc
Average of SON season saved to avg_season/SON_avg/zmla_Eday_CESM2-WACCM.nc


In [16]:
# CESM WACCM FV2 4
urls41 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_20100101-20150101.nc"
    ]

urls42 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_20100101-20150101.nc"
    ]

urls43 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_20100101-20150101.nc"
    ]

urls4 = [urls41,urls42,urls43]

calculate_and_save_seasonal_averages(urls4)

rm: cannot remove '*.nc': No such file or directory
cleared!
zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19500101-19591231.nc download done
zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19600101-19691231.nc download done
zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19700101-19791231.nc download done
zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19800101-19891231.nc download done
zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19900101-19991231.nc download done
zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_20000101-20091231.nc download done
zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_20100101-20150101.nc download done
zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19500101-19591231.nc download done
zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19600101-19691231.nc download done
zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19700101-19791231.nc download done
zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19800101-19891231.nc download done
zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19900101-19991231.nc download done
zml

OSError: ignored

In [ ]:
# EC-Earth3-Veg-LR ! 6

urls61 = [
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19790101-19791231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19800101-19801231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19810101-19811231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19820101-19821231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19830101-19831231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19840101-19841231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19850101-19851231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19860101-19861231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19870101-19871231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19880101-19881231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19890101-19891231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19900101-19901231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19910101-19911231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19920101-19921231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19930101-19931231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19940101-19941231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19950101-19951231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19960101-19961231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19970101-19971231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19980101-19981231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19990101-19991231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20000101-20001231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20010101-20011231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20020101-20021231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20030101-20031231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20040101-20041231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20050101-20051231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20060101-20061231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20070101-20071231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20080101-20081231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20090101-20091231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20100101-20101231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20110101-20111231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20120101-20121231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20130101-20131231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20140101-20141231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20150101-20151231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20160101-20161231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20170101-20171231.nc"
    ]

url6 = [urls61]

calculate_and_save_seasonal_averages(url6)

rm: cannot remove '*.nc': No such file or directory
cleared!
zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19790101-19791231.nc download done
zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19800101-19801231.nc download done
zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19810101-19811231.nc download done
zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19820101-19821231.nc download done
zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19830101-19831231.nc download done
zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19840101-19841231.nc download done
zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19850101-19851231.nc download done
zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19860101-19861231.nc download done
zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19870101-19871231.nc download done
zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19880101-19881231.nc download done
zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19890101-19891231.nc download done
zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19900101-19901231.nc downl

In [ ]:
# INM-CM4-8 ! 8

urls81 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM4-8/amip/r1i1p1f1/Eday/zmla/gr1/v20190528/zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_19790101-19881231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM4-8/amip/r1i1p1f1/Eday/zmla/gr1/v20190528/zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_19890101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM4-8/amip/r1i1p1f1/Eday/zmla/gr1/v20190528/zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_19990101-20081231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM4-8/amip/r1i1p1f1/Eday/zmla/gr1/v20190528/zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_20090101-20141231.nc"
    ]
urls8 = [urls81]

calculate_and_save_seasonal_averages(urls8)

rm: cannot remove '*.nc': No such file or directory
cleared!
zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_19790101-19881231.nc download done
zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_19890101-19981231.nc download done
zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_19990101-20081231.nc download done
zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_20090101-20141231.nc download done
['zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_20090101-20141231.nc', 'zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_19890101-19981231.nc', 'zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_19990101-20081231.nc', 'zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_19790101-19881231.nc']
rm: cannot remove 'avg_season': Is a directory
rm: cannot remove 'sample_data': Is a directory
rm: cannot remove 'season_avg_data_files': Is a directory
rm: cannot remove '.nc': No such file or directory
avg1 done
Average of DJF season saved to avg_season/DJF_avg/zmla_Eday_INM-CM4-8.nc
Average of MAM season saved to avg_season/MAM_avg/zmla_Eday_INM-CM4-8.nc
Average of JJA season saved to avg_sea

In [ ]:
# INM-CM5-0 ! 8

urls91 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM5-0/amip/r1i1p1f1/Eday/zmla/gr1/v20190610/zmla_Eday_INM-CM5-0_amip_r1i1p1f1_gr1_19790101-19881231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM5-0/amip/r1i1p1f1/Eday/zmla/gr1/v20190610/zmla_Eday_INM-CM5-0_amip_r1i1p1f1_gr1_19890101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM5-0/amip/r1i1p1f1/Eday/zmla/gr1/v20190610/zmla_Eday_INM-CM5-0_amip_r1i1p1f1_gr1_19990101-20081231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM5-0/amip/r1i1p1f1/Eday/zmla/gr1/v20190610/zmla_Eday_INM-CM5-0_amip_r1i1p1f1_gr1_20090101-20141231.nc"
    ]

urls9 = [urls91]

calculate_and_save_seasonal_averages(urls9)

In [ ]:
# IPSL-CM6A-LR ! 10

urls101 = [
    "http://vesg.ipsl.upmc.fr/thredds/fileServer/cmip6/CMIP/IPSL/IPSL-CM6A-LR/amip/r2i1p1f1/Eday/zmla/gr/v20181109/zmla_Eday_IPSL-CM6A-LR_amip_r2i1p1f1_gr_19580101-19741231.nc"
]

urls10 = [urls101]

calculate_and_save_seasonal_averages(urls10)

rm: cannot remove '*.nc': No such file or directory
cleared!
zmla_Eday_IPSL-CM6A-LR_amip_r2i1p1f1_gr_19580101-19741231.nc download done
['zmla_Eday_IPSL-CM6A-LR_amip_r2i1p1f1_gr_19580101-19741231.nc']
rm: cannot remove 'avg_season': Is a directory
rm: cannot remove 'sample_data': Is a directory
rm: cannot remove 'season_avg_data_files': Is a directory
rm: cannot remove '.nc': No such file or directory
avg1 done
Average of DJF season saved to avg_season/DJF_avg/zmla_Eday_IPSL-CM6A-LR.nc
Average of MAM season saved to avg_season/MAM_avg/zmla_Eday_IPSL-CM6A-LR.nc
Average of JJA season saved to avg_season/JJA_avg/zmla_Eday_IPSL-CM6A-LR.nc
Average of SON season saved to avg_season/SON_avg/zmla_Eday_IPSL-CM6A-LR.nc


#

In [ ]:
# MIROC-ES2L ! 11

urls111 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20140101-20141231.nc"
    ]


urls112 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20140101-20141231.nc"
    ]

urls119 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20140101-20141231.nc"
    ]
urls11 = [urls111,urls112,urls119]

calculate_and_save_seasonal_averages(urls11)

In [ ]:
import netCDF4 as nc
import numpy as np
import requests
import os
import glob


def calculate_and_save_seasonal_averages1():

    files = glob.glob("*.nc")
    print(files)
    for file in files:
        ds = nc.Dataset(file)
        season_data = ds.variables['zmla']
        lat = ds.variables['lat'][:]
        lon = ds.variables['lon'][:]

        # print(season_data)
        # Get the time variable
        time_var = ds['time']

        # Convert numeric time values to datetime objects
        time_values = time_var[:]
        time_units = time_var.units
        calendar = time_var.calendar if hasattr(
            time_var, 'calendar') else 'standard'

        dates = nc.num2date(time_values, units=time_units, calendar=calendar)

        # Define the start and end months for each season (assuming a calendar year)
        seasons = {
            'DJF': (12, 1, 2),
            'MAM': (3, 4, 5),
            'JJA': (6, 7, 8),
            'SON': (9, 10, 11)
        }

        # Loop through each season and compute the average
        for season, (start_month, mid_month, end_month) in seasons.items():

            # Find indices of dates within the current season
            season_indices = [
                i for i, date in enumerate(dates) if (date.month == start_month or date.month == mid_month or date.month == end_month)
            ]
            season_data = ds.variables['zmla'][season_indices, :, :]

            # Calculate the seasonal average
            seasonal_avg = season_data.mean(axis=0)

            # Create a new netCDF file for the seasonal average
            filename = file

            # create the new file within season name folder having that season's average
            season_avg_file = f'season_avg_data_files/{season}_avg/{filename}'

            ncfile = nc.Dataset(season_avg_file, mode='w', format='NETCDF4')

            dim1 = ncfile.createDimension('dim1', seasonal_avg.shape[0])
            dim2 = ncfile.createDimension('dim2', seasonal_avg.shape[1])

            dim3 = ncfile.createDimension('dim3', lat.shape[0])
            dim4 = ncfile.createDimension('dim4', lon.shape[0])

            var = ncfile.createVariable('average', 'f4', ('dim1', 'dim2'))
            varlat = ncfile.createVariable('lat', 'f4', dim3)
            varlot = ncfile.createVariable('lon', 'f4', dim4)

            # write the data to the variable
            var[:] = seasonal_avg[:]
            varlat[:] = lat[:]
            varlot[:] = lon[:]

            # Close the new netCDF file
            ncfile.close()

        # Close the original netCDF file
        ds.close()

    # remove downloaded files
    !rm * .nc
    print("avg1 done")
    calc_average_seasonal_files1()


def calc_average_seasonal_files1():
    # Create a dictionary to store seasonal averages
    seasonal_averages = {
        'DJF': [],
        'MAM': [],
        'JJA': [],
        'SON': [],
    }

    # Loop through the seasons and calculate the average of files
    for season in seasonal_averages.keys():
        season_files = glob.glob(f"season_avg_data_files/{season}_avg/*.nc")

        if not season_files:
            print(f"No files found for {season} season.")
            continue

        # Create an array to store seasonal averages for this season
        season_averages = []

        for season_file in season_files:
            ds = nc.Dataset(season_file)
            seasonal_avg = ds.variables['average'][:]
            season_averages.append(seasonal_avg)
            lat = ds.variables['lat'][:]
            lon = ds.variables['lon'][:]
            ds.close()

        # Calculate the average of seasonal averages
        average_seasonal_avg = np.mean(season_averages, axis=0)

        # Create the output directory if it doesn't exist
        output_dir = f'avg_season/{season}_avg'
        os.makedirs(output_dir, exist_ok=True)

        # Extract the filename from the original file's path
        original_filename = os.path.basename(season_files[0])
        parts = original_filename.split('_')
        filename = '_'.join(parts[:3])
        filename = filename + ".nc"

        # Create a new netCDF file for the average seasonal average
        avg_season_file = os.path.join(output_dir, filename)


        ncfile = nc.Dataset(avg_season_file, mode='w', format='NETCDF4')

        dim1 = ncfile.createDimension('dim1', average_seasonal_avg.shape[0])
        dim2 = ncfile.createDimension('dim2', average_seasonal_avg.shape[1])
        dim3 = ncfile.createDimension('dim3', lat.shape[0])
        dim4 = ncfile.createDimension('dim4', lon.shape[0])

        var = ncfile.createVariable('average', 'f4', ('dim1', 'dim2'))
        var[:] = average_seasonal_avg[:]

        varlat = ncfile.createVariable('lat', 'f4', 'dim3')
        varlat[:] = lat

        varlon = ncfile.createVariable('lon', 'f4', 'dim4')
        varlon[:] = lon

        ncfile.close()

        print(f"Average of {season} season saved to {avg_season_file}")

        # Clean up temporary seasonal average files
        for season_file in season_files:
            os.remove(season_file)

In [ ]:
calculate_and_save_seasonal_averages1()

['zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19600101-19691231.nc', 'zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19500101-19591231.nc']


<ipython-input-7-000807f73766>:45: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  season_data = ds.variables['zmla'][season_indices, :, :]


rm: cannot remove 'sample_data': Is a directory
rm: cannot remove 'season_avg_data_files': Is a directory
rm: cannot remove '.nc': No such file or directory
avg1 done
Average of DJF season saved to avg_season/DJF_avg/zmla_Eday_CESM2-FV2.nc
Average of MAM season saved to avg_season/MAM_avg/zmla_Eday_CESM2-FV2.nc
Average of JJA season saved to avg_season/JJA_avg/zmla_Eday_CESM2-FV2.nc
Average of SON season saved to avg_season/SON_avg/zmla_Eday_CESM2-FV2.nc
